<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_11_11_Multi_turn_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit to perform multi-step summarization of provided text.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [8]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [9]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [10]:
from typing import List, Optional

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def revise_text(prompt: str, text: str, specification_id: str, conversation_id: Optional[str] = None):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.revise_text(prompt=prompt, text=text, id=conversation_id, specification=input_types.EntityReferenceInput(id=specification_id))

        message = response.revise_text.message.message if response.revise_text is not None and response.revise_text.message is not None else None
        conversation_id = response.revise_text.conversation.id if response.revise_text is not None and response.revise_text.conversation is not None else None

        return message, conversation_id
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)


Execute Graphlit example

In [11]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

text = """Sample Written History and Physical Examination
History and Physical Examination
 Patient Name:
Date:
 Rogers, Pamela
 6/2/04
Referral Source: Emergency Department
Data Source:
Patient
Chief Complaint & ID:  Ms. Rogers is a 56 y/o WF
having chest pains for the last week.
History of Present Illness
 This is the first admission for this 56 year old woman,
who states she was in her usual state of good health until
one week prior to admission.  At that time she noticed the
abrupt onset (over a few seconds to a minute) of chest pain
which she describes as dull and aching in character.  The
pain began in the left para-sternal area and radiated up to
her neck.  The first episode of pain one week ago occurred
when she was working in her garden in the middle of the
day.  She states she had been working for approximately 45
minutes and began to feel tired before the onset of the pain.
Her discomfort was accompanied by shortness of breath, but
no sweating, nausea, or vomiting.  The pain lasted
approximately 5 to 10 minutes and resolved when she went
inside and rested in a cool area.
Since that initial pain one week ago she has had 2 additional
episodes of pain, similar in quality and location to the first
episode.  Three days ago she had a 15 minute episode of
pain while walking her dog, which resolved with rest.  This
evening she had an episode of pain awaken her from sleep,
lasting 30 minutes, which prompted her visit to the
Emergency Department.  At no time has she attempted any
specific measures to relieve her pain, other than rest.  She
describes no other associated symptoms during these
episodes of pain, including dizziness, or palpitations.  She
becomes short of breath during these
episodes but describes no other exertional dyspnea,
orthopnea, or paroxysmal nocturnal dyspnea.  No change in the pain
with movement, no association with food, no GERD sx, no palpable pain.
She has never been told she has heart problems, never had any
chest pains before, does not have claudication.  She was diagnosed with
HTN 3 years ago,
She does not smoke nor does she have diabetes.
She was diagnosed with hypertension 3 years ago and had a
TAH with BSO 6 years ago.  She is not on hormone replacement
therapy.  There is a family history of premature CAD.
She does not know her cholesterol level.
Past Medical History
 Surgical –
1994:
1998:
 Total abdominal hysterectomy and bilateral
oophorectomy for uterine fibroids.
Comments
 Define the reason for the patient’s visit as who has been
specifically as possible.
Convey the acute or chronic nature of the problem and
establish a chronology.
onset
character
location
radiation
circumstances; exacerbating factors
associated symptoms
duration
resolution; alleviating factors
Describe the natural history of her problem since its
onset
Change or new circumstances to the problem
New duration
Reason she come in for visit
What has patient tried for relief
    Relevant positive and negative ROS for this complaint
Review of systems for the relevant organ system
Relevant risk factor/environmental conditions
This highly relevant, although it may seem like a
trivial detail at first
 Bunionectomy

Medical History –
1998: Diagnosed with hypertension and began on
  unknown medication.  Stopped after 6 months
  because of drowsiness.
1990: Diagnosed with peptic ulcer disease, which
  resolved after three months on cimetidine.  She  Always use generic names
  describes no history of cancer, lung disease
or previous heart disease.

Allergy: Penicillin; experienced rash and hives in 1985.  Always list the type of reported reaction

Social History –
Alcohol use: 1 or 2 beers each weekend; 1 glass of   Quantity
   wine once a week with dinner.
Tobacco use: None.
Medications: No prescription or illegal drug use.
   Occasional OTC ibuprofen (Advil) for  Include over-the-counter drugs
   headache (QOD).

Family History

Mother:  79, alive and well.     Comment specifically on the presence or absence of
Father:  54, deceased, heart attack.  No brothers  diseases relevant to the chief complaint
   or sisters.  There is a positive family history of
   hypertension, but no diabetes, or cancer.

Review of Systems

HEENT:
No complaints of headache change in vision, nose or ear   Separate each ROS section for easy identification
problems, or sore throat.

Cadiovascular:
See HPI        OK to refer to HPI if adequately covered there


Gastrointestinal:
No complaints of dysphagia, nausea, vomiting, or change in  List positive and negative findings in brief, concise
stool pattern, consistency, or color.  She complains of   phrases or sentences
epigastric pain, burning in quality, approximately twice a
month, which she notices primarily at night.

Genitourinary:
No complaints of dysuria, nocturia, polyuria, hematuria, or
vaginal bleeding.

Musculoskeletal:
She complains of lower back pain, aching in quality,
approximately once every week after working in her garden.
This pain is usually relieved with Tylenol.  She complains of
no other arthralgias, muscle aches, or pains.

Neurological:
She complains of no weakness, numbness, or incoordination.

Physical Examination
 Vital Signs:
Blood Pressure 168/98, Pulse 90, Respirations 20,
Temperature 37 degrees.
General:
Ms. Rogers appears alert, oriented and cooperative.
Skin:
Normal in appearance, texture, and temperature
HEENT:
Scalp normal.
Pupils equally round, 4 mm, reactive to light and
accommodation, sclera and conjunctiva normal.
Fundoscopic examination reveals normal vessels without
hemorrhage.
Tympanic membranes and external auditory canals normal.
Nasal mucosa normal.
Oral pharynx is normal without erythema or exudate.  Tongue
and gums are normal.
Neck:
Easily moveable without resistance, no abnormal adenopathy
in the cervical or supraclavicular areas.  Trachea is midline
and thyroid gland is normal without masses.  Carotid artery
upstroke is normal bilaterally without bruits.  Jugular venous
pressure is measured as 8 cm with patient at 45 degrees.
Chest:
Lungs are clear to auscultation and percussion bilaterally
except for crackles heard in the lung bases bilaterally.  PMI
is in the 5th inter-costal space at the mid clavicular line.  A
grade 2/6 systolic decrescendo murmur is heard best at the
second right inter-costal space which radiates to the neck.
A third heard sound is heard at the apex.  No fourth heart
sound or rub are heard.  Cystic changes are noted in the
breasts bilaterally but no masses or nipple discharge is
Seen.
Abdomen:
The abdomen is symmetrical without distention; bowel
sounds are normal in quality and intensity in all areas; a
bruit is heard in the right paraumbilical area.  No masses or
splenomegaly are noted; liver span is 8 cm by percussion.
Extremities:
No cyanosis, clubbing, or edema are noted.  Peripheral
pulses in the femoral, popliteal, anterior tibial, dorsalis pedis,
brachial, and radial areas are normal.
Nodes:
No palpable nodes in the cervical, supraclavicular, axillary
or inguinal areas.
Genital/Rectal:
Normal rectal sphincter tone;  no rectal masses or
tenderness.  Stool is brown and guaiac negative.  Pelvic
Always list vital signs.  Check for orthostatic BP/P
     changes if it is relevant to the patient’s complaint
Description may give very important clues as to the
        nature or severity of the patient’s problem
Comment on all organ systems
      List specific normal or pathological findings when
        relevant to the patient’s complaint
This patient needs a detailed cardiac examination
More precise than saying “no hepatomegaly”
Always include these exams, or comment specifically
why they were omitted
exmaination reveals normal external genitalia, and normal
vagina and cervix on speculum examination.  Bimanual
examination reveals no palpable uterus, ovaries, or masses.

Neurological:
Cranial nerves II-XII are normal.  Motor and sensory
examination of the upper and lower extremities is normal.
Gait and cerebellar function are also normal.  Reflexes are
normal and symmetrical bilaterally in both extremities.

Initial Problem List

1. Chest Pain      Although you can omit this initial problem list from your
2. Dyspnea       final written H&P, (and just list a final problem list
3. History of HTN (4 years)     shown below), it is useful to make an initial list simply
4. History of TAH/BSO     to keep track of all problems uncovered in the interview
5. History of peptic ulcer disease    (#1-9 in this list) and exam (#10-13)
6. Penicillin allergy
7. FH of early ASCVD
8. Epigastric pain
9. Low back pain
10. Hypertension
11. Systolic murmur
12. Cystic changes of breasts
13. Abdominal bruit

Revised Problem List

1. Chest pain      This list regroups related problems (or those you
2. FH of early ASCVD     suspect are related) into a more logical sequence
3. Early surgical menopause
4. Dyspnea
5. Recent onset HTN
6. Abdominal bruit
7. Systolic ejection murmur
8. Epigastric pain
9. History of peptic ulcer disease
10. Lumbosacral back pain
11. OTC non-steroidal analgesic use
12. Cystic changes of breasts
13. Penicillin allergy

Assessment and Differential Diagnosis

1. Chest pain with features of angina pectoris

This patient’s description of dull, aching, exertion
related substernal chest pain is suggestive of ischemic cardiac
origin.  Her findings of a FH of early ASCVD, hypertension, and
early surgical menopause are pertinent risk factors for development
of coronary artery disease.  Therefore, the combination of this
patient’s presentation, and the multiple risk factors make angina
pectoris the most likely diagnosis.  The pain symptoms appear to
be increasing, and the occurrence of pain at rest suggests this
fits the presentation of unstable angina, and hospitalization is
indicated.

  Other processes may explain her chest pain, but
are less likely.  Gastro-esophageal reflux disease (GERD) may
occur at night with recumbency, but usually is not associated with
exertion.  The pain of GERD is usually burning, and the patient
describes no associated gastrointestinal symptoms such as
nausea, vomiting or abdominal pain which might suggest peptic
ulcer disease.  The presence of dyspnea might suggest a
pulmonary component to this patient’s disease process, but
the absence of fever, cough or abnormal pulmonary examination
findings make a pulmonary infection less likely, and the
association of the dyspnea with the chest pain supports the
theory that both symptoms may be from ischemic heart disease.

2. Dyspnea

Her dyspnea may correlate with findings on physical
exam of a third heart sound and pulmonary crackles,
suggesting left ventricular dysfunction.  In that case, she may
be manifesting symptoms and findings of congestive heart
failure from myocardial ischemia.

3. Recent onset hypertension and abdominal bruit

This combination raises the possibility of a
secondary cause of hypertension, specifically ASCVD of the
renal artery leading to renovascular hypertension.  The lack
of hypertensive retinopathy and left ventricular hypertrophy
on physical examination further support a recent onset of her
BP elevation.

4. Systolic murmur

The possibility of important valvular heart disease
is raised by the murmur, specifically, aortic stenosis.  The
murmur radiates to the neck as an aortic valvular murmur
often does, but a normal carotid upstroke may mean this
murmur is not significant.

5. Epigastric discomfort, NSAID use with a history
of peptic ulcer disease.

6. Lumbo-sacral back pain

7. Fibrocystic breast disease

8. Penicillin allergy

In the assessment you take each of the patient’s major
problems and draw conclusions, in this case that the chest
pain is more likely due to ischemic heart disease instead
of other possibilities.  You tie in several of the other
problems as risk factors for ischemic heart disease, and
not merely as random unrelated problems.  You should list
and extensive justification for your most likely diagnosis.
You should also explain why you are less suspicious of
alternative diagnoses, such as esophageal reflux disease,
pulmonary or musculoskeletal pain.

As in the previous problem, you should explain, in more
detail than is shown in this example, why you felt the dyspnea
is more likely to be from ischemic heart disease, and not
asthma, bronchitis, or other possibilities.  Follow this pattern
for all subsequent problems.

Plan:

1. Carefully monitor the patient for any increased chest pain that
might be indicative of impending myocardial infarction by admitting
the patient to the telemetry floor.

2. Start platelet inhibitors, such as aspirin to decrease the risk of
myocardial infarction; start nitrates to decrease the risk of occlusion
and to treat her symptoms of pain.  For prolonged pain un-
responsive to nitrates, she may need an analgesic such as
morphine.  The nitrates will also help to lower her BP.

3. Patient should have her cholesterol monitored and when
discharged she should be started on an appropriate exercise and
weight loss program, including a low-fat diet.  If her cholesterol
is elevated, she may need cholesterol-lowering medication such
as HMG Co-reductases.

4. Schedule a cardiac catheterization since non-invasive
tests have a high pretest probability for being positive and regard-
less of the result, negative or positive, she will need a cath

5. Begin diuretics for her dyspnea which is most likely secondary
to volume overload – this will treat her high BP as well.  She should
have a ventriculogram with the cath that will assess cardiac size
and presence of wall motion abnormalities.

6. Appropriate lab work would include BUN/Creatinine
to assess kidney function, electrolytes and baseline EKG.

You should develop a diagnostic and therapeutic plan
for the patient.  Your plan should incorporate acute and
long-term care of the patient’s most likely problem.  You
should consider pharmacologic and non-pharmacologic
measures and be cognizant of the fact that you need to
treat the symptoms (i.e. make the patient comfortable) as
much as treating the disease when possible.  You are
expected to know the usual classes of medications used
to treat these illnesses.
 """

Deleted all conversations and specifications.


Create specification, and start multi-turn summarization conversation.

In [12]:
specification_id = await create_openai_specification(enums.OpenAIModels.O1_MINI_128K)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    prompt = "Summarize this for me."

    message, conversation_id = await revise_text(prompt, text, specification_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


Created specification [4eac23b3-e600-4dc7-9162-666e449a7d2a].


**Revision:**
Pamela Rogers, a 56-year-old woman, presents with a one-week history of intermittent chest pain.

The pain is described as dull and aching, located substernally, radiating to the neck, and is often triggered by exertion.

She has a history of hypertension diagnosed three years ago and underwent total abdominal hysterectomy and bilateral oophorectomy six years ago.

Her family history is significant for premature coronary artery disease.

Physical examination reveals elevated blood pressure, crackles in the lung bases, and a systolic murmur.

Assessment suggests angina pectoris likely due to ischemic heart disease, with a concern for unstable angina.

The management plan includes admission for telemetry, initiation of aspirin and nitrates, cholesterol monitoring, and scheduling cardiac catheterization.

In [13]:
    prompt = "OK, not bad, but make it longer."

    message, conversation_id = await revise_text(prompt, text, specification_id, conversation_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


**Revision:**
Pamela Rogers is a 56-year-old white female who presented to the Emergency Department with a one-week history of intermittent chest pain. Her initial episode occurred while gardening, lasting approximately 5 to 10 minutes, and was accompanied by shortness of breath. Since then, she has experienced additional episodes of similar chest pain, including one three days prior while walking her dog and a recent episode that awakened her from sleep for 30 minutes.

The chest pain is described as dull and aching, located in the left para-sternal area, and radiates to the neck. It is often triggered by exertion and relieved by rest. She denies associated symptoms such as sweating, nausea, vomiting, dizziness, or palpitations. Her medical history is significant for hypertension diagnosed three years ago, for which she discontinued medication due to drowsiness, and a total abdominal hysterectomy with bilateral oophorectomy performed six years ago. She also has a history of peptic ulcer disease resolved with cimetidine.

Family history is notable for premature coronary artery disease, with her father having died of a heart attack at age 54. She does not smoke, consume alcohol excessively, or use illicit drugs. Her review of systems is largely negative except for occasional epigastric pain and lower back pain relieved by Tylenol. Physical examination revealed elevated blood pressure of 168/98 mmHg, pulse of 90 bpm, and crackles at the lung bases. A grade 2/6 systolic decrescendo murmur was heard best at the second right inter-costal space, radiating to the neck.

The assessment indicates angina pectoris likely due to ischemic heart disease, with features suggestive of unstable angina given the recent onset and increasing frequency of chest pain episodes, including pain at rest. Additionally, the presence of a systolic murmur raises concerns for possible valvular heart disease, although the normal carotid upstroke may indicate it is not significant. Her dyspnea and pulmonary crackles suggest possible left ventricular dysfunction and early signs of congestive heart failure.

The management plan involves admitting Ms. Rogers to the telemetry floor for continuous monitoring, initiating aspirin and nitrates to manage her chest pain and reduce the risk of myocardial infarction, and scheduling cardiac catheterization to assess the extent of coronary artery disease. Additionally, her cholesterol levels will be monitored, and she will be advised on lifestyle modifications including diet and exercise. Diuretics may be started to address her dyspnea and elevated blood pressure.

In [14]:
    prompt = "Rewrite from the perspective of a medical content writer. Add additional medical details. Format nicely with Markdown for a blog post."

    message, conversation_id = await revise_text(prompt, text, specification_id, conversation_id)

    if message is not None:
        display(Markdown(f'**Revision:**\n{message}'))
        print()


**Revision:**
# Understanding Angina Pectoris: A Comprehensive Case Study

## Patient Background

Pamela Rogers is a 56-year-old Caucasian woman with a significant medical history including hypertension diagnosed three years ago and a total abdominal hysterectomy with bilateral oophorectomy performed six years prior. Her family history is notable for premature coronary artery disease, as her father succumbed to a heart attack at the age of 54. Ms. Rogers maintains a relatively healthy lifestyle, abstaining from smoking and illicit drug use, and consumes alcohol moderately.

## Clinical Presentation

Ms. Rogers presented to the Emergency Department with a one-week history of intermittent chest pain. The initial episode occurred while gardening, lasting between five to ten minutes, and was accompanied by shortness of breath. She has since experienced additional episodes, including one while walking her dog and another that awakened her from sleep, lasting approximately thirty minutes. The pain is described as dull and aching, originating substernally and radiating to the neck, typically triggered by exertion and relieved by rest. Notably, she denies associated symptoms such as sweating, nausea, or palpitations.

Upon physical examination, Ms. Rogers exhibited elevated blood pressure (168/98 mmHg) and a systolic decrescendo murmur heard best at the second right intercostal space, radiating to the neck. Additionally, crackles were noted at the lung bases, and a third heart sound was present at the apex, suggesting possible left ventricular dysfunction.

## Assessment and Differential Diagnosis

The primary assessment indicates angina pectoris likely due to ischemic heart disease, given the patient's presentation and risk factors such as hypertension and a family history of coronary artery disease. The increasing frequency of chest pain episodes, including pain at rest, raises concern for unstable angina. The presence of a systolic murmur warrants evaluation for potential valvular heart disease, though the lack of a significant carotid upstroke suggests it may be benign.

Differential diagnoses include gastroesophageal reflux disease (GERD), which is less likely due to the exertional nature of the pain and absence of gastrointestinal symptoms. Pulmonary conditions such as infections are also considered but are deemed unlikely given the absence of fever and cough. Musculoskeletal pain is another consideration; however, the radiating nature of the chest pain and associated dyspnea support a cardiac origin.

## Management and Treatment Plan

Ms. Rogers has been admitted to the telemetry unit for continuous cardiac monitoring. Initial management includes the administration of aspirin to inhibit platelet aggregation and nitrates to alleviate chest pain and reduce myocardial oxygen demand. Given her hypertensive state, nitrates also aid in lowering blood pressure. Additionally, cholesterol levels will be assessed, and lifestyle modifications such as a low-fat diet and regular exercise will be recommended to mitigate further cardiovascular risk.

A cardiac catheterization has been scheduled to evaluate the extent of coronary artery disease. This invasive procedure will provide detailed insights into the coronary anatomy and guide potential interventional treatments. Considering her pulmonary findings, diuretics may be initiated to manage possible left ventricular dysfunction and prevent the progression to congestive heart failure.

## Conclusion

This case underscores the importance of a thorough clinical evaluation in patients presenting with chest pain. Recognizing the signs of unstable angina and initiating prompt management can significantly impact patient outcomes. Ongoing monitoring and tailored therapeutic interventions are essential in managing ischemic heart disease and preventing adverse cardiac events.